# Imports

In [ ]:
# ============================================================================
# 03_modeling.ipynb
# Modeling and Evaluation for Diabetes Binary Classification
# Extended with Hyperparameter Tuning, Multiple Models, ANN, and Cost-Benefit Analysis
# Integrated with preprocessing module
# ============================================================================
"""
Notes:
- RandomizedSearchCV is used for efficiency.
- For expensive models (RF, XGBoost) tuning runs on a 30% subsample, then the best estimator is refit on the full training set.
- CV for RandomizedSearchCV is set to cv=5 as requested.
- MLP tuning uses early stopping and reduced max_iter for speed.
- Best models are saved to disk for use in 04_results.py
"""

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import warnings
import joblib
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.neural_network import MLPClassifier

import xgboost as xgb

from scipy.stats import randint, uniform

from pyexpat import features

# Add src directory to Python path to import preprocessing module
project_root = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.insert(0, project_root)

# Import preprocessing module
try:
    from dpp.preprocessing import get_preprocessed_data
    PREPROCESSING_AVAILABLE = True
except ImportError as e:
    PREPROCESSING_AVAILABLE = False
    print(f"Warning: Could not import preprocessing module. Error: {e}")
    print("Using placeholder data for demonstration.")

# Load Preprocessed Data

In [2]:
# ============================================================================
# 1. LOAD PREPROCESSED DATA
# ============================================================================
print("="*80)
print("LOAD PREPROCESSED DATA")
print("="*80)

if PREPROCESSING_AVAILABLE:
    # Load preprocessed data from preprocessing module
    try:
        data = get_preprocessed_data()
        features_train = data['features_train']
        target_train = data['target_train']
        features_test = data['features_test']
        target_test = data['target_test']

        print(f"Training set size: {len(features_train):,}")
        print(f"Test set size: {len(features_test):,}")
        print(f"Number of features: {features_train.shape[1]}")
    except Exception as e:
        print(f"Error loading preprocessed data: {e}")
        print("Using placeholder data for demonstration.")
        PREPROCESSING_AVAILABLE = False
else:
    print("Preprocessing module not available.")

LOAD PREPROCESSED DATA
[preprocessing] Loading data from: C:\Users\Eyyub\Desktop\StackFuel\PortfolioProjekt\DPP-Stackfuel-Data-Science-Projekt\data\raw\diabetes-health-indicators-dataset\diabetes_binary_health_indicators_BRFSS2015.csv
Training set size: 311,002
Test set size: 45,895
Number of features: 24


# Model Initialization

In [3]:
# ============================================================================
# 2. MODEL INITIALIZATION (base estimators)
# ============================================================================
print("\n" + "="*80)
print("MODEL INITIALIZATION (BASE ESTIMATORS)")
print("="*80)

base_estimators = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'XGBoost': xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    # Use LinearSVC as base; wrap in CalibratedClassifierCV later if you need probabilities
    'SVM': CalibratedClassifierCV(LinearSVC(random_state=42, max_iter=10000, dual=False), cv=5),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Naive Bayes': GaussianNB(),
    'Neural Network': MLPClassifier(random_state=42, max_iter=500)
}



MODEL INITIALIZATION (BASE ESTIMATORS)


# Hyperparameter Tuning

In this section, hyperparameter tuning is performed only for four selected models:
Random Forest, XGBoost, Logistic Regression, and Neural Network.

The rationale behind this selective tuning approach is based on a balance between 
computational efficiency and expected impact on model performance:

1. Effectiveness of Hyperparameters:
   These four models have hyperparameters that significantly influence their 
   predictive performance. For example, the number of trees and depth in Random Forest,
   learning rate and tree depth in XGBoost, regularization strength in Logistic Regression,
   and architecture parameters in Neural Networks can greatly affect results.

2. Computational Cost:
   Exhaustive hyperparameter tuning (e.g., GridSearchCV) is computationally expensive,
   especially when applied to many models with large parameter grids. Focusing on 
   models with the highest potential gain optimizes resource usage.

3. Simplicity and Speed of Other Models:
   Other models like Decision Trees, K-Nearest Neighbors, Naive Bayes, and Linear SVM 
   typically have fewer or less impactful hyperparameters, or are inherently faster to train.
   Their default parameters often provide reasonable baseline performance.

4. Practical Workflow:
   This approach allows for a manageable and efficient modeling pipeline, prioritizing 
   tuning efforts where they are most likely to yield substantial improvements.

If desired, hyperparameter tuning can be extended to additional models using more 
efficient search strategies (e.g., RandomizedSearchCV) or by tuning a smaller subset 
of parameters to balance performance gains with computational cost.


In [4]:
# ============================================================================
# 3. HYPERPARAMETER TUNING (RandomizedSearchCV, CV=5)
# ============================================================================
print("\n" + "="*80)
print("HYPERPARAMETER TUNING")
print("Note: Tuning for expensive models (RF, XGB) runs on a 30% subsample to save time.")
print("="*80)

# Create a 30% subsample for expensive model tuning
features_sub, _, target_sub, _ = train_test_split(features_train, target_train, train_size=0.30, stratify=target_train, random_state=42)

# Parameter distributions for RandomizedSearch
param_distributions = {
    'Random Forest': {
        'n_estimators': [50, 100, 150, 200],
        'max_depth': [8, 12, 20, None],
        'min_samples_split': [2, 5, 10],
        'max_features': ['sqrt', 'log2', 0.5]
    },
    'XGBoost': {
        'n_estimators': [100, 200],
        'max_depth': [3, 6],
        'learning_rate': [0.05, 0.1, 0.2],
        'subsample': [0.7, 0.8, 1.0]
    },
    'Logistic Regression': {
        'C': [0.01, 0.1, 1, 10],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear'],
        'class_weight': [None, 'balanced']
    },
    'Neural Network': {
        'hidden_layer_sizes': [(50,), (100,), (50, 50)],
        'activation': ['relu', 'tanh'],
        'alpha': [0.0001, 0.001],
        'learning_rate': ['constant', 'adaptive']
    }
}

# We will store the final trained models here
trained_models = {}




HYPERPARAMETER TUNING
Note: Tuning for expensive models (RF, XGB) runs on a 30% subsample to save time.


In [ ]:
# Helper function to run RandomizedSearchCV and refit best estimator on full training set
def tune_and_refit(name, estimator, param_dist, X_tune, y_tune, X_full, y_full, n_iter=20, cv=5):
    """
    Tune estimator with RandomizedSearchCV on X_tune/y_tune (subsample for speed),
    then refit the best estimator on the full training data X_full/y_full.
    Returns the refitted best estimator and the best CV score.
    """
    print(f"[tuning] {name}: Starting RandomizedSearchCV (n_iter={n_iter}, cv={cv}) ...")
    rs = RandomizedSearchCV(
        estimator=estimator,
        param_distributions=param_dist,
        n_iter=n_iter,
        scoring='roc_auc',
        cv=cv,
        random_state=42,
        n_jobs=-1,
        verbose=1,
        refit=True
    )
    t0 = time.time()
    rs.fit(X_tune, y_tune)
    t1 = time.time()
    print(f"[tuning] {name}: RandomizedSearchCV finished in {(t1-t0)/60:.2f} min. Best CV score: {rs.best_score_:.4f}")
    print(f"[tuning] {name}: Best params: {rs.best_params_}")

    # Refit best estimator on full training data for final model
    best_est = rs.best_estimator_
    print(f"[tuning] {name}: Refitting best estimator on the full training set ...")
    t2 = time.time()
    best_est.fit(X_full, y_full) # pyright: ignore[reportAttributeAccessIssue]
    t3 = time.time()
    print(f"[tuning] {name}: Refit finished in {(t3-t2)/60:.2f} min.")
    return best_est, rs.best_score_



In [ ]:
# Tuning loop for selected models
tuned_models = {}
tuned_scores = {}

# Random Forest tuning (on subsample, then refit on full)
if 'Random Forest' in base_estimators:
    rf_est = RandomForestClassifier(random_state=42, n_jobs=-1)
    rf_param_dist = param_distributions['Random Forest']
    best_rf, best_rf_score = tune_and_refit('Random Forest', rf_est, rf_param_dist, features_sub, target_sub, features_train, target_train, n_iter=20, cv=5)
    trained_models['Random Forest'] = best_rf
    tuned_scores['Random Forest'] = best_rf_score



[tuning] Random Forest: Starting RandomizedSearchCV (n_iter=20, cv=5) ...
Fitting 5 folds for each of 20 candidates, totalling 100 fits
[tuning] Random Forest: RandomizedSearchCV finished in 14.17 min. Best CV score: 0.9598
[tuning] Random Forest: Best params: {'n_estimators': 100, 'min_samples_split': 5, 'max_features': 'log2', 'max_depth': None}
[tuning] Random Forest: Refitting best estimator on the full training set ...
[tuning] Random Forest: Refit finished in 0.59 min.


In [7]:
# XGBoost tuning (if available)
if 'XGBoost' in param_distributions:
    xgb_est = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss', n_jobs=-1)
    xgb_param_dist = param_distributions['XGBoost']
    best_xgb, best_xgb_score = tune_and_refit('XGBoost', xgb_est, xgb_param_dist, features_sub, target_sub, features_train, target_train, n_iter=20, cv=5)
    trained_models['XGBoost'] = best_xgb
    tuned_scores['XGBoost'] = best_xgb_score



[tuning] XGBoost: Starting RandomizedSearchCV (n_iter=20, cv=5) ...
Fitting 5 folds for each of 20 candidates, totalling 100 fits
[tuning] XGBoost: RandomizedSearchCV finished in 2.18 min. Best CV score: 0.9635
[tuning] XGBoost: Best params: {'subsample': 0.7, 'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.1}
[tuning] XGBoost: Refitting best estimator on the full training set ...
[tuning] XGBoost: Refit finished in 0.16 min.


In [8]:
# Logistic Regression tuning
if 'Logistic Regression' in param_distributions:
    lr_est = LogisticRegression(random_state=42, max_iter=1000)
    lr_param_dist = param_distributions['Logistic Regression']
    best_lr, best_lr_score = tune_and_refit('Logistic Regression', lr_est, lr_param_dist, features_sub, target_sub, features_train, target_train, n_iter=20, cv=5)
    trained_models['Logistic Regression'] = best_lr
    tuned_scores['Logistic Regression'] = best_lr_score



[tuning] Logistic Regression: Starting RandomizedSearchCV (n_iter=20, cv=5) ...
Fitting 5 folds for each of 16 candidates, totalling 80 fits
[tuning] Logistic Regression: RandomizedSearchCV finished in 23.56 min. Best CV score: 0.8135
[tuning] Logistic Regression: Best params: {'solver': 'liblinear', 'penalty': 'l2', 'class_weight': None, 'C': 0.1}
[tuning] Logistic Regression: Refitting best estimator on the full training set ...
[tuning] Logistic Regression: Refit finished in 0.05 min.


In [10]:
# --- Neural Network tuning (MLP), FASTER VERSION ---
# Use a 30% subsample for tuning, enable early stopping, reduce max_iter, and reduce n_iter
if 'Neural Network' in param_distributions:
    # Subsample for MLP tuning
    X_sub_nn, _, y_sub_nn, _ = train_test_split(features_train, target_train, train_size=0.30, stratify=target_train, random_state=42)

    # Base estimator with early stopping and reduced iterations for tuning
    nn_est = MLPClassifier(random_state=42, max_iter=200, early_stopping=True, n_iter_no_change=10)

    nn_param_dist = param_distributions['Neural Network']

    # Randomized search with fewer iterations (n_iter=10) and cv=5
    best_nn, best_nn_score = tune_and_refit(
        'Neural Network',
        nn_est,
        nn_param_dist,
        X_sub_nn,
        y_sub_nn,
        features_train,
        target_train,
        n_iter=10,   # smaller search budget for speed
        cv=5
    )
    trained_models['Neural Network'] = best_nn
    



[tuning] Neural Network: Starting RandomizedSearchCV (n_iter=10, cv=5) ...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[tuning] Neural Network: RandomizedSearchCV finished in 17.54 min. Best CV score: 0.9196
[tuning] Neural Network: Best params: {'learning_rate': 'constant', 'hidden_layer_sizes': (100,), 'alpha': 0.001, 'activation': 'relu'}
[tuning] Neural Network: Refitting best estimator on the full training set ...
[tuning] Neural Network: Refit finished in 1.96 min.


In [11]:
# For any remaining models not tuned, fit defaults on the full training set
for name, est in base_estimators.items():
    if name in trained_models:
        continue
    if est is None:
        continue
    print(f"[fit-default] Fitting default estimator for {name} on full training set ...")
    t0 = time.time()
    est.fit(features_train, target_train)
    t1 = time.time()
    print(f"[fit-default] {name} fitted in {(t1-t0)/60:.2f} min.")
    trained_models[name] = est


[fit-default] Fitting default estimator for SVM on full training set ...
[fit-default] SVM fitted in 0.23 min.
[fit-default] Fitting default estimator for K-Nearest Neighbors on full training set ...
[fit-default] K-Nearest Neighbors fitted in 0.00 min.
[fit-default] Fitting default estimator for Decision Tree on full training set ...
[fit-default] Decision Tree fitted in 0.08 min.
[fit-default] Fitting default estimator for Naive Bayes on full training set ...
[fit-default] Naive Bayes fitted in 0.00 min.


# Model Evaluation

In [12]:
# ============================================================================
# 4. MODEL EVALUATION
# ============================================================================
print("\n" + "="*80)
print("MODEL EVALUATION")
print("="*80)

model_results = {}
for name, model in trained_models.items():
    print(f"\nEvaluating {name} ...")
    # Ensure model supports predict_proba; if not, try to obtain calibrated probabilities
    try:
        y_proba = model.predict_proba(features_test)[:, 1]
    except Exception:
        # fallback: use decision_function and min-max scale (heuristic) - ideally calibrate with holdout
        if hasattr(model, "decision_function"):
            scores = model.decision_function(features_test)
            from sklearn.preprocessing import MinMaxScaler
            y_proba = MinMaxScaler().fit_transform(scores.reshape(-1, 1)).ravel()
        else:
            # last-resort: use predict (not ideal)
            y_proba = model.predict(features_test)

    y_pred = (y_proba >= 0.5).astype(int) if y_proba.ndim == 1 else model.predict(features_test)

    auc_score = roc_auc_score(target_test, y_proba)
    cv_scores = cross_val_score(model, features_train, target_train, cv=5, scoring='roc_auc', n_jobs=-1)

    model_results[name] = {
        'model': model,
        'y_pred': y_pred,
        'y_proba': y_proba,
        'auc': auc_score,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std()
    }

    print(f"AUC Score: {auc_score:.4f}")
    print(f"CV AUC Score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# Identify best model
best_model_name = max(model_results.keys(), key=lambda k: model_results[k]['auc'])
best_model = model_results[best_model_name]['model']
best_y_proba = model_results[best_model_name]['y_proba']

print(f"\nBest model: {best_model_name} with AUC: {model_results[best_model_name]['auc']:.4f}")


MODEL EVALUATION

Evaluating Random Forest ...
AUC Score: 0.7887
CV AUC Score: 0.9751 (+/- 0.0906)

Evaluating XGBoost ...
AUC Score: 0.8153
CV AUC Score: 0.9511 (+/- 0.1847)

Evaluating Logistic Regression ...
AUC Score: 0.8078
CV AUC Score: 0.8155 (+/- 0.0115)

Evaluating Neural Network ...
AUC Score: 0.7897
CV AUC Score: 0.9234 (+/- 0.1606)

Evaluating SVM ...
AUC Score: 0.8075
CV AUC Score: 0.8152 (+/- 0.0116)

Evaluating K-Nearest Neighbors ...
AUC Score: 0.7070
CV AUC Score: 0.9198 (+/- 0.0120)

Evaluating Decision Tree ...
AUC Score: 0.5901
CV AUC Score: 0.8469 (+/- 0.2513)

Evaluating Naive Bayes ...
AUC Score: 0.7647
CV AUC Score: 0.7708 (+/- 0.0120)

Best model: XGBoost with AUC: 0.8153
